In [41]:
import pandas as pd
import numpy as np
 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MinMaxScaler

from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import SMOTE

In [2]:
data = pd.read_csv('MPB-Dataset_Clean.csv')

In [3]:
data.describe()

,target,floor,m2,bathrooms,toilets,shower_rooms,number_bedrooms,primary_energy_consumption,co2_emission
count,3240.000000,3240.000000,3240.000000,3240.000000,3240.000000,3240.000000,3240.000000,3240.000000,3240.000000
mean,1708.300617,2.841667,111.975617,1.030864,1.470988,1.104938,1.981790,142.436420,26.739198
std,870.549462,2.262202,49.662986,0.561421,0.696942,0.339951,0.906892,162.762193,36.225600
min,140.000000,0.000000,35.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,1120.000000,1.000000,75.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,1450.000000,2.000000,101.000000,1.000000,1.000000,1.000000,2.000000,117.000000,17.000000
75%,2000.000000,4.000000,130.000000,1.000000,2.000000,1.000000,3.000000,214.000000,42.000000
max,6000.000000,12.000000,300.000000,3.000000,4.000000,3.000000,5.000000,1352.000000,296.000000


# Train-Test Split

In [4]:
y = data['target']
X = data.drop('target', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [6]:
X_train_cat = X_train.select_dtypes(include = np.object)
X_test_cat = X_test.select_dtypes(include = np.object)
X_train_num = X_train.select_dtypes(include = np.number)
X_test_num = X_test.select_dtypes(include = np.number)

C:\Users\PC\AppData\Local\Temp\ipykernel_11648\2694347417.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train_cat = X_train.select_dtypes(include = np.object)
C:\Users\PC\AppData\Local\Temp\ipykernel_11648\2694347417.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test_cat = X_test.select_dtypes(include = np.object)


# Treat Categorical

In [7]:
encoder = OneHotEncoder(drop='first').fit(X_train_cat)

In [9]:
X_train_cat_encoded_array = encoder.transform(X_train_cat).toarray()
X_train_cat_treated = pd.DataFrame(X_train_cat_encoded_array, columns = encoder.get_feature_names_out(input_features = X_train_cat.columns ))

X_test_cat_encoded_array = encoder.transform(X_test_cat).toarray()
X_test_cat_treated = pd.DataFrame(X_test_cat_encoded_array, columns = encoder.get_feature_names_out(input_features = X_train_cat.columns ))

# Treat Numerical

In [10]:
num_transformer = MinMaxScaler().fit(X_train_num)

In [12]:
X_train_num_transformed_array = num_transformer.transform(X_train_num)
X_train_num_treated = pd.DataFrame(X_train_num_transformed_array, columns = X_train_num.columns)

X_test_num_transformed_array = num_transformer.transform(X_test_num)
X_test_num_treated = pd.DataFrame(X_test_num_transformed_array, columns = X_test_num.columns)

# display(X_train_num_treated)
# display(X_test_num_treated)

# Concatenate

In [26]:
X_train = pd.concat([X_train_num_treated, X_train_cat_treated], axis=1)
X_test = pd.concat([X_test_num_treated, X_test_cat_treated], axis=1)

display(X_train)
display(X_test)

,floor,m2,bathrooms,toilets,shower_rooms,number_bedrooms,primary_energy_consumption,co2_emission,municipality_bruxelles,municipality_etterbeek,...,postcode_1140,postcode_1180,postcode_1190,postcode_Other,double_glazing_Yes,swimming_pool_Yes,dining_room_Yes,laundry_room_Yes,basement_Yes,air_conditioning_Yes
0,0.500000,0.075472,0.333333,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.166667,0.124528,0.333333,0.333333,0.0,0.25,0.031805,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.416667,0.316981,0.333333,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.500000,0.256604,0.333333,0.333333,0.0,0.25,0.181953,0.179054,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.000000,0.867925,0.333333,0.666667,0.5,0.75,0.121302,0.118243,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,0.166667,0.335849,0.333333,0.333333,0.0,0.50,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
2588,0.000000,0.339623,0.666667,0.333333,0.5,0.25,0.241864,0.236486,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2589,0.000000,0.547170,0.666667,0.666667,0.5,0.50,0.197485,0.179054,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2590,0.416667,0.188679,0.333333,0.000000,0.0,0.25,0.221893,0.000000,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0


,floor,m2,bathrooms,toilets,shower_rooms,number_bedrooms,primary_energy_consumption,co2_emission,municipality_bruxelles,municipality_etterbeek,...,postcode_1140,postcode_1180,postcode_1190,postcode_Other,double_glazing_Yes,swimming_pool_Yes,dining_room_Yes,laundry_room_Yes,basement_Yes,air_conditioning_Yes
0,0.166667,0.132075,0.333333,0.000000,0.0,0.00,0.071746,0.067568,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.166667,0.283019,0.666667,0.333333,0.0,0.50,0.051036,0.043919,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.333333,0.320755,0.333333,0.000000,0.0,0.25,0.148669,0.135135,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.333333,0.264151,0.333333,0.333333,0.0,0.50,0.051036,0.047297,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.000000,0.245283,0.333333,0.333333,0.0,0.25,0.156065,0.189189,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,0.416667,0.226415,0.333333,0.000000,0.0,0.25,0.102811,0.118243,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
644,0.250000,0.698113,1.000000,0.333333,0.0,0.50,0.152367,0.138514,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
645,0.083333,0.245283,0.333333,0.333333,0.0,0.00,0.142751,0.128378,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
646,0.250000,0.320755,0.333333,0.333333,0.0,0.25,0.076183,0.070946,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0


# Linnear Regression

In [22]:
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

In [25]:
predictions_lm = lm.predict(X_test)
r2_score(y_test, predictions_lm)

0.6313554069670984

In [37]:
mae = mean_absolute_error(y_test, predictions_lm)
mae

353.0657859303414

In [38]:
rmse = np.sqrt(mean_squared_error(y_test,predictions_lm))
rmse

483.62112975223215

RMSE is more affected by outliers

In [29]:
KNNR = KNeighborsRegressor()
KNNR.fit(X_train,y_train)

predictions_KNNR = KNNR.predict(X_test)
r2_score(y_test, predictions_KNNR)

0.5081068430748017

In [30]:
MLPR = MLPRegressor()
MLPR.fit(X_train,y_train)

predictions_MLPR = MLPR.predict(X_test)
r2_score(y_test, predictions_MLPR)

C:\Users\PC\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.4006156507767231